In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

I = cv.imread('Picture2.png').astype(float)/255

plt.imshow(I)

In [ ]:
L = cv.Laplacian(I, cv.CV_64F)
plt.imshow(L)

h = np.sqrt(L[:,:,0]**2 + L[:,:,1]**2 + L[:,:,2]**2)

plt.imshow(h)

In [ ]:
Gx = cv.Sobel(I, cv.CV_64F, 1, 0)
Gy = cv.Sobel(I, cv.CV_64F, 0, 1)
Gmag = np.sqrt(Gx**2 + Gy**2)
plt.imshow(Gmag)
plt.show()

g = np.sqrt(Gmag[:,:,0]**2 + Gmag[:,:,1]**2 + Gmag[:,:,2]**2)

plt.imshow(g)

In [ ]:
Gxx = cv.Sobel(I, cv.CV_64F, 2, 0)
Gyy = cv.Sobel(I, cv.CV_64F, 0, 2)
Lmag = np.sqrt(Gxx**2 + Gyy**2)
plt.imshow(Lmag)
plt.show()

l = np.sqrt(Lmag[:,:,0]**2 + Lmag[:,:,1]**2 + Lmag[:,:,2]**2)

plt.imshow(l)
plt.show()
plt.imshow(h)

In [ ]:
def wrap(m):
    return (m + np.pi) % (2 * np.pi) - np.pi

gx = np.sqrt(Gx[:,:,0]**2 + Gx[:,:,1]**2 + Gx[:,:,2]**2)
gy = np.sqrt(Gy[:,:,0]**2 + Gy[:,:,1]**2 + Gy[:,:,2]**2)
gxx = np.sqrt(Gxx[:,:,0]**2 + Gxx[:,:,1]**2 + Gxx[:,:,2]**2)
gyy = np.sqrt(Gyy[:,:,0]**2 + Gyy[:,:,1]**2 + Gyy[:,:,2]**2)


Gdir1 = wrap(2*np.arctan2(Gy[:,:,1], Gx[:,:,1]))
plt.imshow(Gdir1)
plt.show()

# Gdir2 = wrap(4*np.arctan2(Gyy[:,:,1], Gxx[:,:,1]))
# plt.imshow(Gdir2)
# plt.show()

# Gdir_diff = wrap(Gdir2 - Gdir1)

g1mag = np.sqrt(gx**2 + gy**2)
g2mag = np.sqrt(gxx**2 + gyy**2)

K2 = 4

# gdir = wrap(Gdir1 + K2/(1+K2)*Gdir_diff)
gdir = Gdir1
gmag = np.sqrt(g1mag**2 + (K2*g2mag)**2)

plt.imshow(gmag)
plt.show()

plt.imshow(gdir)
plt.colorbar()
plt.show()


In [ ]:
def colorize(mag, angle):
    hsv = np.ones((mag.shape[0], mag.shape[1], 3), dtype=np.float32)
    hsv[:,:,0] = (angle+np.pi)*180/np.pi
    hsv[:,:,2] = mag/2
    return cv.cvtColor(hsv, cv.COLOR_HSV2RGB)

plt.imshow(colorize(gmag, gdir))

In [ ]:
def get_offset(angle):
    if angle < -3*np.pi/4:
        return -1, 0
    if angle < -np.pi/4:
        return -1, 1
    if angle < np.pi/4:
        return 0, 1
    if angle < 3*np.pi/4:
        return 1, 1
    return 1, 0

def nms(mag, angle):
    out = np.ones_like(mag)
    for i in range(1, mag.shape[0]-1):
        for j in range(1, mag.shape[1]-1):
            di, dj = get_offset(angle[i,j])
            if mag[i+di, j+dj] > mag[i, j] or mag[i-di, j-dj] > mag[i, j]:
                out[i, j] = 0
    return out

out = nms(gmag, gdir)

plt.imshow(gmag*out)
plt.show()

plt.imshow(colorize(gmag*out, gdir))
plt.show()


In [ ]:
Gx = cv.Scharr(I, cv.CV_64F, 1, 0)
Gy = cv.Scharr(I, cv.CV_64F, 0, 1)

G45 = 2*Gx*Gy / np.sqrt(Gx**2+Gy**2+0.001)
G0 = (Gx**2 - Gy**2) / np.sqrt(Gx**2+Gy**2+0.001)


g45 = cv.GaussianBlur(G45[:,:,0] + G45[:,:,1] + G45[:,:,2], (5,5), 0)
g0 = cv.GaussianBlur(G0[:,:,0] + G0[:,:,1] + G0[:,:,2], (5,5), 0)

plt.imshow(g45)
plt.show()

plt.imshow(g0)
plt.show()

gmag = np.sqrt(g0**2 + g45**2)
gdir = np.arctan2(g45, g0)

plt.imshow(colorize(gmag, gdir))
plt.show()


out = nms(gmag, gdir)
plt.imshow(colorize(gmag*out, gdir))
plt.show()

In [ ]:
e = out*gmag

thresh_high = 2
thresh_low = 0.3

plt.imshow(e > thresh_high)
plt.show()

plt.imshow(e > thresh_low)
plt.show()

strong = e >= thresh_high
weak = e >= thresh_low

keep = np.copy(strong)

for i in range(1, keep.shape[0]-1):
    for j in range(1, keep.shape[1]-1):
        if weak[i,j] and (keep[i,j] or keep[i,j+1] or keep[i,j-1] or keep[i-1,j] or keep[i-1,j+1] or keep[i-1,j-1] or keep[i+1,j] or keep[i+1,j+1] or keep[i+1,j-1]):
            keep[i,j] = 1

plt.imshow(keep)
plt.show()

s = np.zeros((keep.shape[0], keep.shape[1], 3))
s[:,:,0] = strong
s[:,:,1] = keep
s[:,:,2] = weak
plt.imshow(s)
plt.show()

In [ ]:
def find_anchors(g0, g45):
    anchors = []
    thresh = 0.4
    for i in range(1, g0.shape[0]-1, 8):
        for j in range(1, g0.shape[1]-1):
            if g0[i,j] > g0[i,j+1] + thresh and g0[i,j] > g0[i,j-1] + thresh:
                anchors.append((j,i))
    for i in range(1, g0.shape[0]-1):
        for j in range(1, g0.shape[1]-1, 8):
            if g0[i,j] < g0[i+1,j] - thresh and g0[i,j] < g0[i-1,j] - thresh:
                anchors.append((j,i))
    thresh = thresh*1.414
    for i in range(1, g0.shape[0]-1, 3):
        for j in range(1, g0.shape[1]-1, 3):
            if g45[i,j] > g45[i+1,j+1] + thresh and g45[i,j] > g45[i-1,j-1] + thresh:
                anchors.append((j,i))
            if g45[i,j] < g45[i-1,j+1] - thresh and g45[i-1,j] < g45[i+1,j-1] - thresh:
                anchors.append((j,i))
    return anchors

anchors = find_anchors(g0, g45)

anchor_map = np.zeros_like(g0)
for anchor in anchors:
    anchor_map[anchor[1], anchor[0]] = 1

print(len(anchors))
plt.imshow(cv.dilate(anchor_map, cv.getStructuringElement(cv.MORPH_RECT, (3,3), (1, 1))))
# plt.imshow(anchor_map[0:200,600:800])
# plt.imshow(anchor_map)
plt.show()            
    

In [ ]:
visited_map = np.zeros_like(g0, dtype=bool)

thresh = 0.

gmag[0,:] = 0
gmag[-1,:] = 0
gmag[:,0] = 0
gmag[:,-1] = 0

di1_lut = np.zeros((3,3,2,2), dtype=int)
dj1_lut = np.zeros((3,3,2,2), dtype=int)
di2_lut = np.zeros((3,3,2,2), dtype=int)
dj2_lut = np.zeros((3,3,2,2), dtype=int)

for di in range(-1,2):
    for dj in range(-1,2):
        for g0_pos in range(2):
            for g45_pos in range(2):
                if g0_pos:
                    di1, dj1 = (1, 0)
                else:
                    di1, dj1 = (0, 1)
                if g45_pos > 0:
                    di2, dj2 = (di1-dj1, dj1-di1)
                else:
                    di2, dj2 = (1, 1)

                if (di1+di2)*di + (dj1+dj2)*dj > 0:
                    di1_lut[di+1, dj+1, g0_pos, g45_pos] = di1
                    di2_lut[di+1, dj+1, g0_pos, g45_pos] = di2
                    dj1_lut[di+1, dj+1, g0_pos, g45_pos] = dj1
                    dj2_lut[di+1, dj+1, g0_pos, g45_pos] = dj2
                else:
                    di1_lut[di+1, dj+1, g0_pos, g45_pos] = -di1
                    di2_lut[di+1, dj+1, g0_pos, g45_pos] = -di2
                    dj1_lut[di+1, dj+1, g0_pos, g45_pos] = -dj1
                    dj2_lut[di+1, dj+1, g0_pos, g45_pos] = -dj2

segments = []
scores = []

for anchor in anchors:
    segment = []
    score = 0
    j, i = anchor
    di = -1
    dj = -1
    while(gmag[i,j] > thresh and not visited_map[i, j]):
        segment.append((j,i))
        score += 1#gmag[i,j]
        visited_map[i, j] = True
        g0_pos = 1 if g0[i,j] > 0 else 0
        g45_pos = 1 if g45[i,j] > 0 else 0

        di1 = di1_lut[di+1, dj+1, g0_pos, g45_pos]
        dj1 = dj1_lut[di+1, dj+1, g0_pos, g45_pos]
        di2 = di2_lut[di+1, dj+1, g0_pos, g45_pos]
        dj2 = dj2_lut[di+1, dj+1, g0_pos, g45_pos]

        if gmag[i + di1,j + dj1] >= gmag[i + di2, j + dj2]:
            di,dj = (di1,dj1)
        else:
            di,dj = (di2,dj2)

        i += di
        j += dj
    j, i = anchor
    di = 1
    dj = 1
    if score > 0:
        visited_map[i, j] = False
        segment.reverse()
        del segment[-1]
        score -= 1#gmag[i,j]
        while(gmag[i,j] > thresh and not visited_map[i, j]):
            segment.append((j,i))
            score += 1#gmag[i,j]
            visited_map[i, j] = True
            g0_pos = 1 if g0[i,j] > 0 else 0
            g45_pos = 1 if g45[i,j] > 0 else 0

            di1 = di1_lut[di+1, dj+1, g0_pos, g45_pos]
            dj1 = dj1_lut[di+1, dj+1, g0_pos, g45_pos]
            di2 = di2_lut[di+1, dj+1, g0_pos, g45_pos]
            dj2 = dj2_lut[di+1, dj+1, g0_pos, g45_pos]

            if gmag[i + di1,j + dj1] >= gmag[i + di2, j + dj2]:
                di,dj = (di1,dj1)
            else:
                di,dj = (di2,dj2)

            i += di
            j += dj

    segments.append(segment)
    scores.append(score)

# for anchor in anchors:
#     j, i = anchor
#     di = -1
#     dj = -1
#     while(gmag[i,j] > thresh and not visited_map[i, j]):
#         visited_map[i, j] = True
#         if g0[i,j] > 0:
#             di1, dj1 = (1, 0)
#         else:
#             di1, dj1 = (0, 1)
#         if g45[i,j] > 0:
#             di2, dj2 = (di1-dj1, dj1-di1)
#         else:
#             di2, dj2 = (1, 1)
#         i1 = i+di1
#         j1 = j+dj1
#         i2 = i+di2
#         j2 = j+dj2

#         if gmag[i1,j1] >= gmag[i2,j2]:
#             i,j = (i1,j1)
#         else:
#             i,j = (i2,j2)


plt.imshow(visited_map)
plt.show()
plt.imshow(gmag+visited_map*10)
plt.show()


sorted_index = sorted(range(len(scores)), key=lambda k: -scores[k])
laser_map = np.zeros_like(g0, dtype=bool)
for k in range(20):
    for pt in segments[sorted_index[k]]:
        laser_map[pt[1], pt[0]] = 1

plt.imshow(laser_map)
plt.show()

In [ ]:

plt.imshow(1-I[:,:,0])
plt.show()
plt.imshow(I[:,:,1])
plt.show()
plt.imshow(I[:,:,2])
plt.show()

In [ ]:
Z = np.zeros_like(I)
a11 = 0
a12 = 1
a13 = 1
a21 = 0
a22 = 0
a23 = 1
a31 = 1
a32 = 1
a33 = 1
Z[:,:,0] = a11*I[:,:,2] + a13*(1-I[:,:,0]) + a12*4*np.abs(cv.Laplacian(I[:,:,1], cv.CV_64F))
Z[:,:,1] = a21*I[:,:,2] + a23*(1-I[:,:,0]) + a22*4*np.abs(cv.Laplacian(I[:,:,1], cv.CV_64F))
Z[:,:,2] = a31*I[:,:,2] + a33*(1-I[:,:,0]) + a32*4*np.abs(cv.Laplacian(I[:,:,1], cv.CV_64F))

plt.imshow(Z)
plt.show()

In [ ]:
Z = np.zeros_like(Gmag)
#fluff
a11 = 0
a21 = 0
a31 = 1

#tentacles
a12 = .3
a22 = 1
a32 = .3

#body
a13 = 1
a23 = 0
a33 = 0

Iswap = np.zeros_like(I)
Iswap[:,:,0] = I[:,:,2]
Iswap[:,:,1] = I[:,:,1]
Iswap[:,:,2] = I[:,:,0]

hpf = Iswap - cv.GaussianBlur(Iswap, None, 25)
# plt.imshow(hpf+0.5)
# plt.show()
# plt.imshow(1-2*hpf)
# plt.show()
# plt.imshow(2*hpf)
# plt.show()


plt.imshow(hpf[:,:,0]+0.5)
plt.show()

plt.imshow(hpf[:,:,1]+0.5)
plt.show()

plt.imshow(hpf[:,:,2]+0.5)
plt.show()

#z3 = np.clip(4*(0.5-I[:,:,1]), 0,1)
#z3 = np.clip(8*cv.Laplacian(I[:,:,1], cv.CV_64F), 0,1)
#z3 = 4*cv.Laplacian(I[:,:,1], cv.CV_64F)
z1 = 2*hpf[:,:,0]

#z2 = (1-I[:,:,0])
z2 = -2*hpf[:,:,1]

#z1 = (I[:,:,2])
z3 = -2*hpf[:,:,2]



Z[:,:,0] = a11*z1 + a12*z2 + a13*z3
Z[:,:,1] = a21*z1 + a22*z2 + a23*z3
Z[:,:,2] = a31*z1 + a32*z2 + a33*z3

plt.imshow(Z)
plt.show()

z1 = np.clip(z1, 0,1)
z2 = np.clip(z2, 0,1)
z3 = np.clip(z3, 0,1)


Z[:,:,0] = a11*z1 + a12*z2 + a13*z3
Z[:,:,1] = a21*z1 + a22*z2 + a23*z3
Z[:,:,2] = a31*z1 + a32*z2 + a33*z3

plt.imshow(Z)
plt.show()

In [ ]:
hsv = np.zeros_like(I, dtype=np.float32)

hsv[:,:,0] =  np.mod(4*180/np.pi*np.arctan2(hpf[:,:,0], -hpf[:,:,2])+360, 360)
hsv[:,:,1] =  1
hsv[:,:,2] =  2*(2*hpf[:,:,0] - 2*hpf[:,:,1] - hpf[:,:,2])

# plt.imshow(hsv[:,:,0])
# plt.show()
# plt.imshow(hsv[:,:,1])
# plt.show()

plt.imshow(hsv[:,:,2])
plt.show()

iii = cv.cvtColor(hsv, cv.COLOR_HSV2RGB)

plt.imshow(iii)
plt.show()
